<a href="https://colab.research.google.com/github/tejaspradhan/Graph-Neural-Networks/blob/main/hiv-project/gcn_hiv_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installations

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#@title
# Install rdkit
import sys
import os
import requests
import subprocess
import shutil
from logging import getLogger, StreamHandler, INFO
 
 
logger = getLogger(__name__)
logger.addHandler(StreamHandler())
logger.setLevel(INFO)
 
 
def install(
        chunk_size=4096,
        file_name="Miniconda3-latest-Linux-x86_64.sh",
        url_base="https://repo.continuum.io/miniconda/",
        conda_path=os.path.expanduser(os.path.join("~", "miniconda")),
        rdkit_version=None,
        add_python_path=True,
        force=False):
    """install rdkit from miniconda
    ```
    import rdkit_installer
    rdkit_installer.install()
    ```
    """
 
    python_path = os.path.join(
        conda_path,
        "lib",
        "python{0}.{1}".format(*sys.version_info),
        "site-packages",
    )
 
    if add_python_path and python_path not in sys.path:
        logger.info("add {} to PYTHONPATH".format(python_path))
        sys.path.append(python_path)
 
    if os.path.isdir(os.path.join(python_path, "rdkit")):
        logger.info("rdkit is already installed")
        if not force:
            return
 
        logger.info("force re-install")
 
    url = url_base + file_name
    python_version = "{0}.{1}.{2}".format(*sys.version_info)
 
    logger.info("python version: {}".format(python_version))
 
    if os.path.isdir(conda_path):
        logger.warning("remove current miniconda")
        shutil.rmtree(conda_path)
    elif os.path.isfile(conda_path):
        logger.warning("remove {}".format(conda_path))
        os.remove(conda_path)
 
    logger.info('fetching installer from {}'.format(url))
    res = requests.get(url, stream=True)
    res.raise_for_status()
    with open(file_name, 'wb') as f:
        for chunk in res.iter_content(chunk_size):
            f.write(chunk)
    logger.info('done')
 
    logger.info('installing miniconda to {}'.format(conda_path))
    subprocess.check_call(["bash", file_name, "-b", "-p", conda_path])
    logger.info('done')
 
    logger.info("installing rdkit")
    subprocess.check_call([
        os.path.join(conda_path, "bin", "conda"),
        "install",
        "--yes",
        "-c", "rdkit",
        "python==3.7.3",
        "rdkit" if rdkit_version is None else "rdkit=={}".format(rdkit_version)])
    logger.info("done")
 
    import rdkit
    logger.info("rdkit-{} installation finished!".format(rdkit.__version__))
 
 
if __name__ == "__main__":
    install()

add /root/miniconda/lib/python3.7/site-packages to PYTHONPATH
python version: 3.7.12
fetching installer from https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
done
installing miniconda to /root/miniconda
done
installing rdkit
done
rdkit-2020.09.1 installation finished!


In [3]:
! pip install -q torch-scatter -f https://pytorch-geometric.com/whl/torch-1.10.0+cu111.html
! pip install -q torch-sparse -f https://pytorch-geometric.com/whl/torch-1.10.0+cu111.html
! pip install -q torch-cluster -f https://pytorch-geometric.com/whl/torch-1.10.0+cu111.html
! pip install -q torch-geometric

     |████████████████████████████████| 7.9 MB 4.3 MB/s 
     |████████████████████████████████| 3.5 MB 4.5 MB/s 
     |████████████████████████████████| 2.5 MB 4.2 MB/s 
     |████████████████████████████████| 407 kB 4.2 MB/s 


## Importing Libraries

In [4]:
import rdkit
from rdkit import Chem
from rdkit.Chem import Draw
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch 
import torch.nn.functional as F 
import torch_geometric
from torch_geometric.data import Dataset, Data
from torch_geometric.nn import GATConv, Linear, TopKPooling
from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
from torch_geometric.loader import DataLoader
import numpy as np
from sklearn.metrics import accuracy_score
from tqdm import tqdm
from torch_geometric.nn import GCNConv

## Creating Custom Graph Dataset from CSV File

**Important**
Before running the next cell create a directory structure as follows : 

        |- data
              |- raw
                    |- HIV.csv (data csv file)

In [5]:
class HIVDataset(Dataset):
    def __init__(self, root, filename, test=False, transform=None, pre_transform=None):
        """
        root = Where the dataset should be stored. This folder is split
        into raw_dir (downloaded dataset) and processed_dir (processed data). 
        """
        self.test = test
        self.filename = filename
        super(HIVDataset, self).__init__(root, transform, pre_transform)
        
    @property
    def raw_file_names(self):
        """ If this file exists in raw_dir, the download is not triggered.
            (The download func. is not implemented here)  
        """
        return self.filename

    @property
    def processed_file_names(self):
        return 'not_implemented.pt'

    def download(self):
        pass

    def process(self):
        self.data = pd.read_csv(self.raw_paths[0])
        print(self.raw_paths)
        for index, mol in tqdm(self.data.iterrows(), total=self.data.shape[0]):
            mol_obj = Chem.MolFromSmiles(mol["smiles"])
            # Get node features
            node_feats = self._get_node_features(mol_obj)
            # Get edge features
            edge_feats = self._get_edge_features(mol_obj)
            # Get adjacency info
            edge_index = self._get_adjacency_info(mol_obj)
            # Get labels info
            label = self._get_labels(mol["HIV_active"])

            # Create data object
            data = Data(x=node_feats, 
                        edge_index=edge_index,
                        edge_attr=edge_feats,
                        y=label,
                        smiles=mol["smiles"]
                        ) 
            if self.test:
                torch.save(data, 
                    os.path.join(self.processed_dir, 
                                 f'data_test_{index}.pt'))
            else:
                torch.save(data, 
                    os.path.join(self.processed_dir, 
                                 f'data_{index}.pt'))

    def _get_node_features(self, mol):
        """ 
        This will return a matrix / 2d array of the shape
        [Number of Nodes, Node Feature size]
        """
        all_node_feats = []

        for atom in mol.GetAtoms():
            node_feats = []
            # Feature 1: Atomic number        
            node_feats.append(atom.GetAtomicNum())
            # Feature 2: Atom degree
            node_feats.append(atom.GetDegree())
            # Feature 3: Formal charge
            node_feats.append(atom.GetFormalCharge())
            # Feature 4: Hybridization
            node_feats.append(atom.GetHybridization())
            # Feature 5: Aromaticity
            node_feats.append(atom.GetIsAromatic())
            # Feature 6: Total Num Hs
            node_feats.append(atom.GetTotalNumHs())
            # Feature 7: Radical Electrons
            node_feats.append(atom.GetNumRadicalElectrons())
            # Feature 8: In Ring
            node_feats.append(atom.IsInRing())
            # Feature 9: Chirality
            node_feats.append(atom.GetChiralTag())

            # Append node features to matrix
            all_node_feats.append(node_feats)

        all_node_feats = np.asarray(all_node_feats)
        return torch.tensor(all_node_feats, dtype=torch.float)

    def _get_edge_features(self, mol):
        """ 
        This will return a matrix / 2d array of the shape
        [Number of edges, Edge Feature size]
        """
        all_edge_feats = []

        for bond in mol.GetBonds():
            edge_feats = []
            # Feature 1: Bond type (as double)
            edge_feats.append(bond.GetBondTypeAsDouble())
            # Feature 2: Rings
            edge_feats.append(bond.IsInRing())
            # Append node features to matrix (twice, per direction)
            all_edge_feats += [edge_feats, edge_feats]

        all_edge_feats = np.asarray(all_edge_feats)
        return torch.tensor(all_edge_feats, dtype=torch.float)

    def _get_adjacency_info(self, mol):
        """
        We could also use rdmolops.GetAdjacencyMatrix(mol)
        but we want to be sure that the order of the indices
        matches the order of the edge features
        """
        edge_indices = []
        for bond in mol.GetBonds():
            i = bond.GetBeginAtomIdx()
            j = bond.GetEndAtomIdx()
            edge_indices += [[i, j], [j, i]]

        edge_indices = torch.tensor(edge_indices)
        edge_indices = edge_indices.t().to(torch.long).view(2, -1)
        return edge_indices

    def _get_labels(self, label):
        label = np.asarray([label])
        return torch.tensor(label, dtype=torch.int64)

    def len(self):
        return self.data.shape[0]

    def get(self, idx):
        """ - Equivalent to __getitem__ in pytorch
            - Is not needed for PyG's InMemoryDataset
        """
        if self.test:
            data = torch.load(os.path.join(self.processed_dir, 
                                 f'data_test_{idx}.pt'))
        else:
            data = torch.load(os.path.join(self.processed_dir, 
                                 f'data_{idx}.pt'))   
        return data

In [6]:
train_dataset = HIVDataset(root='data/',filename='HIV_train_oversampled.csv')
test_dataset = HIVDataset(root='data/',filename='HIV_test.csv')

Processing...


['data/raw/HIV_train_oversampled.csv']


100%|██████████| 71634/71634 [03:49<00:00, 311.62it/s]
Done!
Processing...


['data/raw/HIV_test.csv']


100%|██████████| 3999/3999 [00:10<00:00, 391.40it/s]
Done!


In [7]:
train_dataset

HIVDataset(71634)

In [8]:
test_dataset

HIVDataset(3999)

In [9]:
train_dataset[0].x.shape
# 9 node features

torch.Size([19, 9])

In [10]:
train_dataset[0].y.shape

torch.Size([1])

In [11]:
train_dataset[0].edge_index.t().shape # 2 edge featurs 

torch.Size([40, 2])

## Constructing the GCN for this Dataset 

In [12]:
class GCN(torch.nn.Module):
    def __init__(self, feature_size, hidden_channels):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        num_classes = 2
        self.conv1 = GCNConv(feature_size, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, hidden_channels)
        self.conv3 = GCNConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, num_classes)

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)

        # 2. Readout layer
        x = gap(x, batch)  # [batch_size, hidden_channels]

        # 3. Apply a final classifier
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)
        
        return x

##Training GCN


In [13]:
model = GCN(train_dataset[0].x.shape[1], 256)

In [ ]:
model.to('cuda')

GCN(
  (conv1): GCNConv(9, 256)
  (conv2): GCNConv(256, 256)
  (conv3): GCNConv(256, 256)
  (lin): Linear(256, 2, bias=True)
)

In [21]:
# Defining loss and optimiser 
weights = torch.tensor([1,10],dtype=torch.float32).to('cpu')
loss_fn = torch.nn.CrossEntropyLoss(weight = weights)
optimizer = torch.optim.SGD(model.parameters(), lr=0.1,momentum=0.9)
lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer,gamma=0.95)

In [16]:
BATCH_SIZE = 256
EPOCHS=80

In [17]:
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
def train(model, train_loader, optimizer, loss_fn):
    # Enumerate over the data
    all_preds = []
    all_labels = []
    running_loss = 0.0
    step = 0
    for _, batch in enumerate(tqdm(train_loader)):
        batch.to('cuda')  
        optimizer.zero_grad() 
        # forward pass
        pred = model(batch.x.float(), batch.edge_index, batch.batch) 
        # backward pass 
        loss = loss_fn(pred, batch.y)
        loss.backward()  
        optimizer.step()  

        # Update tracking
        running_loss += loss.item()
        step += 1
        all_preds.append(np.argmax(pred.cpu().detach().numpy(),axis=1))
        all_labels.append(batch.y.cpu().detach().numpy())
    all_preds = np.concatenate(all_preds).ravel()
    all_labels = np.concatenate(all_labels).ravel()
    # print(all_preds)
    acc= accuracy_score(all_labels, all_preds)
    return running_loss/step, acc

In [ ]:
losses = []
accs = []
for epoch in range(EPOCHS):
  loss, accuracy = train(model, train_loader, optimizer,loss_fn)
  losses.append(loss)
  accs.append(accuracy)
  if epoch%10==0:
    torch.save(model.state_dict(), "/content/drive/MyDrive/GNN_HIV/models/gcn/model-hiv-"+str(epoch+170) +"epochs.pt")
  print(f"Epoch : {epoch+1} , Loss : {loss}, Accuracy : {accuracy}")

100%|██████████| 280/280 [00:44<00:00,  6.32it/s]


Epoch : 1 , Loss : 0.23617627189627716, Accuracy : 0.5997291788815367


100%|██████████| 280/280 [00:43<00:00,  6.41it/s]


Epoch : 2 , Loss : 0.23484112689537662, Accuracy : 0.6019487952648184


100%|██████████| 280/280 [00:44<00:00,  6.36it/s]


...

Epoch : 78 , Loss : 0.21758842085089003, Accuracy : 0.6524834575760113


100%|██████████| 280/280 [00:41<00:00,  6.72it/s]


Epoch : 79 , Loss : 0.21626544275454113, Accuracy : 0.6553033475723818


100%|██████████| 280/280 [00:42<00:00,  6.57it/s]

Epoch : 80 , Loss : 0.21756227474127496, Accuracy : 0.6529022531200268


In [ ]:
torch.save(model.state_dict(), "/content/drive/MyDrive/GNN_HIV/models/gcn/model-hiv-250epochs.pt")

## Testing 

In [14]:
model.load_state_dict(torch.load('/content/drive/MyDrive/GNN_HIV/models/gcn/model-hiv-500epochs.pt',map_location = torch.device('cpu')))

<All keys matched successfully>

In [22]:
model.eval()
with torch.no_grad():
  test_preds = []
  test_labels = []
  running_loss = 0.0
  steps =0 
  for _, batch in enumerate(tqdm(train_loader)):
    batch.to('cpu')
    pred = model(batch.x.float(),batch.edge_index, batch.batch)
    # print(batch.y)
    loss = loss_fn(pred, batch.y)
    test_preds.append(np.argmax(pred.cpu().detach().numpy(),axis=1))
    test_labels.append(batch.y.cpu().detach().numpy())
    running_loss += loss.item()
    steps+=1
  test_preds = np.concatenate(test_preds).ravel()
  test_labels = np.concatenate(test_labels).ravel()
  acc= accuracy_score(test_labels, test_preds)
  print(f'\nTest Loss : {running_loss/steps}')

100%|██████████| 280/280 [01:45<00:00,  2.67it/s]


Test Loss : 0.13925306624067682


In [24]:
print(acc)

0.8284753050227546
